In [ ]:
url_tmpl = "https://issue.pbsbiotech.com/projects/%s/issues.csv?utf8=%%E2%%9C%%93&columns=all"
_p_urls = [
    "pbscustomer", "pbsdisposables", "pbsinstruments", 
    "magic-metals", "manufacturing", "pbssoftware", "swtesting",
    "system-qualification-testing"
]
project_urls = [url_tmpl % p for p in _p_urls]
project_urls

In [661]:
""" Issuetracker API 

* TODO: Create IssueList class (?)
* Parse Gantt HTML for class 'issue-subject' using style:width to determine hierarchy
* Consider method of lazy evaluation of issue field generation by
calling back to API to download project issues CSV, and update all issues
in project. 
* Implement issue caching

Issue():
    * Add programmatic logging of all fields seen, ever.
    * Map fields seen to types and conversion functions

"""

import requests
import urllib
import pyquery
from collections import OrderedDict
import re
import dateutil.parser
import lxml

uj = urllib.parse.urljoin
_sp_re = re.compile(r"(\d*?) (subproject)?(s{0,1})")
_name2id_re = re.compile(r"(.*?)\s*?#(\d*)$")

class IssuetrackerAPI():
    _login_url = "/login"
    _proj_issues_url = "/projects/%s/issues"
    _issues_url = "/issues"
    _proj_url = "/projects"
    
    def __init__(self, base_url, username=None, pw=None):
        r = urllib.parse.urlparse(base_url)
        if not r.scheme and not r.netloc:
            base_url = urllib.parse.urlunparse(("https", r.path, "", r.params, r.query, r.fragment))
        self._base_url = base_url
        self._sess = requests.Session()
        
        if username is None or pw is None:
            raise ValueError("Must have valid username and password.")
        
        self._username = username
        self._password = pw
        self._auth = (username, pw)
        
        self._cache = {}
        
    def copy(self):
        cls = self.__class__
        new = cls(self._base_url, self._username, self._password)
        return new
    
    def issues(self):
        raise NotImplemented
        
    def projects(self, project_id=None):
        """ Return dict of projects if project_id is None, or project matching 
        `project_id`.
        """
        pj = self._cache.get("projects")
        if not pj:
            pj = self.download_projects()
            self._cache['projects'] = pj
        
        if project_id is not None:
            for attr in ("id", "name", "identifier"):
                for p in pj.values():
                    if getattr(p, attr) == project_id:
                        return p
        return pj
            
    def login(self):
        r1 = self._sess.get(self._base_url)
        r1.raise_for_status()
        q = pyquery.PyQuery(r1.content)
        data = {}
        for td in q("#login-form :input"):
            at = td.attrib
            if 'name' in at:
                k = at['name']
                v = at.get('value', "")
                data[k] = v
                
        data['username'] = self._username
        data['password'] = self._password
        
        body = urllib.parse.urlencode(data)
        r2 = self._sess.post(uj(self._base_url, self._login_url), body)
        r2.raise_for_status()
        if not pyquery.PyQuery(r2.content)("#loggedas"):
            raise ValueError("Invalid Username or Password")
        return r2
        
    def download_project_issues_csv(self, project, utf8=True, columns='all'):
        r = self._download_project_csv(project, utf8, columns)
        return self._parse_proj_csv(r.content)
    
    def _download_project_csv(self, project, utf8, columns):
        if utf8:
            utf8 = "%E2%9C%93"
        else:
            utf8 = ""
        url_end = ".csv?utf8=%s&columns=%s" 
        url = (self._proj_issues_url + url_end) % (project, utf8, columns)
        url = uj(self._base_url, url)
        r = self._sess.get(url)
        r.raise_for_status()
        return r
    
    def download_issue_pdf(self, id):
        href = self._issues_url + "/" + str(id)
        return self.download_issue_pdf2(href)

    def download_issue_pdf2(self, href):
        """ Sometimes it is more convenient to access issue by provided 
        href. """
        type = ".pdf"
        url = uj(self._base_url, href + type)
        r = self._sess.get(url)
        r.raise_for_status()
        return r.content
        
    def _parse_proj_csv(self, csv, encoding='utf-8'):
        if not isinstance(csv, str):
            csv = csv.decode(encoding)
        sl = csv.splitlines()
        sl[0] = sl[0].lower().replace('"', "")
        lines = [l.split(",") for l in sl]
        issues = OrderedDict()
        for i, l in enumerate(lines[1:], 1):
            issue = _Issue(line=sl[i], api=self)
            for key, val in zip(lines[0], l):
                issue[key] = val.strip('"') or "<n/a>"
            issue['#'] = int(issue['#'])
            issues[issue['#']] = issue
        return issues
    
#     def download_projects(self):
#         url = uj(self._base_url, self._proj_url)
#         r = self._sess.get(url)
#         r.raise_for_status()
#         c = r.content
#         q = pyquery.PyQuery(c)
#         q2 = q("#projects-index > [class='projects root']")
#         projects = _Project(self, "All", "")
#         for e in q2.children(".root"):
#             proj_ele = pyquery.PyQuery(e).children(".root > a")[0]
#             pt = proj_ele.text
#             phref = proj_ele.attrib['href'].split("/")[-1]
#             proj = projects.add(pt, phref)
#             q4 = pyquery.PyQuery(e).children("[class='more collapsed']")
#             if len(q4) and _sp_re.match(q4[0].text):
#                 q3 = pyquery.PyQuery(e)("[class='projects ']")
#                 for e2 in q3(".child > .child > a"):
#                     proj.add(e2.text, e2.attrib['href'].split("/")[-1])
#         return projects

    def download_projects(self):
        url = uj(self._base_url, self._proj_url + ".xml")
        print("Downloading projects...")
        r = self._sess.get(url, auth=self._auth)
        r.raise_for_status()
        xml = lxml.etree.XML(r.content)
        projects = {}
        for proj in xml.findall("project"):
            p = Project.from_element(self, proj)
            projects[p.name] = p
        
        # Second pass, process project subtasks
        for p in projects.values():
            if p.parent is not None:
                parent = projects[p.parent['name']]
                parent.add_subproject(p)
        return projects
    
    def _download_gantt_raw(self, project):
        url = (self._proj_issues_url % project) + "/gantt"
        url = uj(self._base_url, url)
        r1 = self._sess.get(url)
        r1.raise_for_status()
        return r1.content

    def download_gantt(self, project):
        project = self.projects(project).identifier
        c = _download_gantt_raw(self, project)
        q = pyquery.PyQuery(c)
        q2 = q(".gantt_subjects")
        i_list = []
        for el in q2.children(".issue-subject"):
            title = el.attrib['title']
            e2=pyquery.PyQuery(el).children("span > a")[0]
            tracker, id = _name2id_re.match(e2.text).groups()
            id = int(id)
            i_list.append(id)
        
        project_issues = self.download_issues(project)
        rv = [] 
        for i in i_list:
            rv.append(project_issues[i])
        
        # sanity check list of subissues. There should be no cycles in this graph. 
        # Also this seems to run in a few ms, so no big deal.
        # _map_issues(rv)
        
        return rv
    
    def _download_project_issues_iter(self, ops, limit, offset):
        ops['limit'] = limit
        ops['offset'] = offset
        url = uj(self._base_url, self._issues_url + ".json")
        url += "?" + urllib.parse.urlencode(ops)
        r = self._sess.get(url, auth=self._auth)
        r.raise_for_status()
        return r

    def download_issues(self, project_id=None, created_on=None, modified_on=None):
        ops = {}
        if project_id:
            if isinstance(project_id, str):
                project_id = self.projects(project_id).id
            ops['project_id'] = project_id
            
        # Unfortunately the api for querying dates and ranges is 
        # quite awkward to translate into a sensible python api
        
        if created_on:
            if not isinstance(created_on, str):
                raise TypeError("Argument created_on must be type str- try .isoformat() (got type %r)" % created_on)
            ops['created_on'] = created_on
            
        if modified_on:
            if not isinstance(modified_on, str):
                raise TypeError("Argument created_on must be type str- try .isoformat() (got type %r)" % modified_on)
            ops['modified_on'] = modified_on
            
        issues = {i.id: i for i in self._download_issues(ops)}
        for iss in issues.values():
            if iss.parent is not None:
                id = iss.parent
                iss.parent = issues[id]
                issues[id].subtasks.append(iss)
        return issues           
            

    def _download_issues(self, ops):
        offset = 0
        limit = 100
        limit = min(max(limit, 0), 100)
        total_count = 0
        
        
        print("\rDownloading issues...", end="")
        while True:
            r = self._download_project_issues_iter(ops, limit, offset)
            d = json.loads(r.content.decode())
            issues = d['issues']

            if not issues:
                break

            yield from self._parse_issues(issues)

            total_count = int(d.get('total_count',0))
            offset += len(issues)
            print("\rDownloading issues: %d/%d      " % (offset, total_count), end="")
            if offset >= total_count:
                break
        print()

    def _parse_issues(self, issues):
        for i in issues:
            yield Issue.from_json(self, **i)
    

def _gantt_duplicate(issue, level):
    raise ValueError("Found duplicate issue at level %d: %r" % (level, issue.subject))

def _map_issues_recursive(issues, seen, level, set_issues):
    for i in issues:
        if i.parent and i.parent in set_issues:
            continue
        if i in seen:
            if level == 0:
                continue
            _gantt_duplicate(issue, level)
        seen.add(i)
        subt = i.subtasks
        _map_issues_recursive(subt, seen, level+1)
    return seen

def _map_issues(issues):
    try:
        seen = _map_issues_recursive(issues, set(), 0, set(issues))
    except ValueError as e:
        e2 = ValueError()
        e2.args = e.args
        raise e2 from None
    if len(seen) != len(issues):
        raise ValueError("Internal error checking gantt integrity: len(seen) != len(issues).")
    if (set(issues) - seen):
        raise ValueError("Internal error checking gantt integrity: Not all issues seen.")

    
def _parse_custom_fields(e):
    rv = {}
    for cf in e.findall("custom_field"):
        cfd = {}
        cfd.update(cf.attrib)
        v = cf.find("value")
        if v is None or v.text == 'blank':
            val = None
        else:
            val = v.text
        cfd['value'] = val
        rv[cfd['name']] = cfd
    return rv

def _parse_datetime(e):
    return dateutil.parser.parse(e.text)

def _parse_int(e):
    return int(e.text)

def _parse_bool(e):
    t = e.text.lower()
    if t == 'false':
        return False
    return True

def _parse_parent(e):
    return {k:v for k,v in e.attrib.items()}


class Project():
    def __init__(self, api, id=0, name="", identifier="", description="", parent=None, status=None, 
                 is_public=False, custom_fields=None, created_on=None, updated_on=None):
        self._api = api
        self.name = name
        self.id = id
        self.name = name
        self.identifier = identifier
        self.description = description
        self.parent = parent
        self.status = status
        self.is_public = is_public
        self.custom_fields = custom_fields
        self.created_on = created_on
        self.updated_on = updated_on
        self._subprojects = []
        
        
    def add_subproject(self, sp):
        sp.parent = self
        if sp not in self._subprojects:
            self._subprojects.append(sp)
            
    def __repr__(self):
        return "_Project(%s)" % ', '.join("%s=%r" % (k[0], getattr(self, k[0])) for k in self._proj_parse_table)
    
    def download_issues(self, utf8=True, columns='all'):
        return self._api.download_project_issues(self.identifier, utf8, columns)
    
    def download_gantt(self):
        return self._api.download_gantt(self.identifier)
        
    _proj_parse_table = [
        # e.tag attr parse function
        ("id", "id", _parse_int),
        ("name", "name", None),
        ("identifier", "identifier", None),
        ("description", "description", None),
        ("parent", "parent", _parse_parent),
        ("status", "status", _parse_int),
        ("is_public", "is_public", _parse_bool),
        ("custom_fields", "custom_fields", _parse_custom_fields),
        ("created_on", "created_on", _parse_datetime),
        ("updated_on", "updated_on", _parse_datetime),
    ]
        
    @classmethod
    def from_element(cls, api, e):
        kw = {}
        for tag, k, func in cls._proj_parse_table:
            el = e.find(tag)
            if el is None:
                continue
            if func:
                v = func(el)
            else:
                v = el.text
            if v is not None:
                kw[k] = v
        if not kw and e.tag != 'project':
            raise ValueError("Failed to parse element: element should be <project> element.")
        return cls(api, **kw)
        
def _unrecognized_kw(kw):
    return ValueError("Unrecognized keywords: %s" % (', '.join(repr(s) for s in kw)))

def _iss_parse_datetime(api, a, v):
    return dateutil.parser.parse(v)

def _iss_parse_int(api, a, v):
    return int(v)

def _iss_parse_usr(api, a, v):
    name = v.pop('name')
    id = v.pop('id')
    if v:
        raise _unrecognized_kw(v)
    return User(api, name, id)

def _iss_parse_resource(api, a, v):
    name = v.pop('name')
    id = v.pop('id')
    value = v.pop('value', "")
    if v:
        raise _unrecognized_kw(v)
    return ResourceWithID(api, name, id, value)

def _iss_parse_project(api, a, v):
    return api.projects()[v['name']]

def _iss_parse_parent(api, a, v):
    if v:
        return int(v['id'])

def _parse_custom_fields(api, a, v):
    fields = {}
    for d in v:
        name = d.pop('name')
        id = d.pop('id')
        val = d.pop('value', "")
        if d:
            raise _unrecognized_kw(d)
        r = ResourceWithID(api, name, id, val)
        fields[name] = val
    return fields


class ResourceWithID():
    def __init__(self, api, name, id, value=""):
        self.api = api
        self.name = name
        self.id = id
        self.value = value
        
    def __repr__(self):
        n = self.__class__.__name__
        args = ', '.join("%s=%r" % (a, getattr(self, a)) for a in ("name", 'id'))
        return "%s(%s)" % (n, args)


class User(ResourceWithID):
    def __init__(self, api, name, id):
        super().__init__(api, name, id)
        del self.value


        
        
class Issue():
    
    _issue_parse_tbl = [
        ("author", "author", _iss_parse_usr),
        ("custom_fields", "custom_fields", _parse_custom_fields),
        ("fixed_version", "sprint_milestone", _iss_parse_resource),  # oddly named. TODO double check this
        ("category", "category", None),
        ("status", "status", _iss_parse_resource),
        ("company", "company", None),
        ("created_on", "created_on", _iss_parse_datetime),
        ("description", "description", None),
        ("subject", "subject", None),
        ("done_ratio", "done_ratio", None),
        ("crm_reply_token", "crm_reply_token", None),
        ("updated_on", "updated_on", _iss_parse_datetime),
        ("id", "id", _iss_parse_int),
        ("project", "project", _iss_parse_project),
        ("contact", "contact", None),
        ("priority", "priority", _iss_parse_resource),
        ("due_date", "due_date", _iss_parse_datetime),
        ("estimated_hours", "estimated_hours", None),
        ("tracker", "tracker", _iss_parse_resource),
        ("parent", "parent", _iss_parse_parent),
        ("closed_on", "closed_on", _iss_parse_datetime),
        ("start_date", "start_date", _iss_parse_datetime),
        ("tracking_uri", "tracking_uri", None),
        ("assigned_to", "assigned_to", _iss_parse_usr)
    ]

    def __init__(self, api, author=None, custom_fields=None, sprint_milestone=None, category=None, status=None, 
                  company=None, created_on=None, description=None, subject=None, done_ratio=None, crm_reply_token=None, 
                  updated_on=None, id=None, project=None, contact=None, priority=None, due_date=None, estimated_hours=None, 
                  tracker=None, parent=None, closed_on=None, start_date=None, tracking_uri=None, assigned_to=None):
        
        self._api = api
        
        self.author = author
        self.custom_fields = custom_fields
        self.sprint_milestone = sprint_milestone  
        self.category = category
        self.status = status
        self.company = company
        self.created_on = created_on
        self.description = description
        self.subject = subject
        self.done_ratio = done_ratio
        self.crm_reply_token = crm_reply_token
        self.updated_on = updated_on
        self.id = id
        self.project = project
        self.contact = contact
        self.priority = priority
        self.due_date = due_date
        self.estimated_hours = estimated_hours
        self.tracker = tracker
        self.parent = parent
        self.closed_on = closed_on
        self.start_date = start_date
        self.tracking_uri = tracking_uri
        self.assigned_to = assigned_to
        
        self.subtasks = []
        
    def __eq__(self, other):
        if not isinstance(other, self.__class__):
            return NotImplemented
        na1 = object()
        na2 = object()
        for _, attr, _ in self._issue_parse_tbl:
            v1 = getattr(self, attr, na1)
            v2 = getattr(other, attr, na2)
            if v1 != v2:
                return False
        return True
                

    @classmethod
    def from_json(cls, api, **kw):
        dct = {}
        absent = object()
        for k, attr, func in cls._issue_parse_tbl:
            v = kw.pop(k, absent)
            if v is absent or not v:
                continue
            if func:
                v = func(api, attr, v)
            dct[attr] = v
        if kw:
            raise _unrecognized_kw(kw)
        return cls(api, **dct)
    
    def add_subtask(self, issue):
        self.subtasks.append(issue)

    def pretty_print(self):
        
        def reprify(v):
            if isinstance(v, self.__class__):
                return "%s(...)" % self.__class__.__name__
            if isinstance(v, dict):
                return "{...}"
            if isinstance(v, list):
                return "[...]"
            return repr(v)
        
        attrs = [t[1] for t in self._issue_parse_tbl]
        args = ", ".join("%s=%s" % (a, reprify(getattr(self, a))) for a in attrs)
        if not args: args = '<empty>'
        return "Issue(%s)" % args

    def download(self, type='pdf'):
        return self._api.download_issue_pdf(self.id)
    
    __str__ = __repr__ = pretty_print


In [483]:
def test_filter(i):
    d=i.download_issues('pbssoftware', ">=2016-08-10")
    def iter5(ob):
        rv=[];ap=rv.append
        for _ in range(5):
            ap(next(ob,None))
        return rv
    import collections; exhaust = collections.deque(maxlen=0).extend
    l = list(d)
    import datetime
    d2 = datetime.datetime(2016, 8, 10).date()
    for item in l:
        co = item.created_on.date()
        mark = "<" if co < d2 else ">" if co > d2 else "="
        print("Issue #%d" % item.id, item.created_on, mark, d2)

In [481]:
import threading
import queue
import time
import os

def download_worker(api, in_q, out_q, path):
    api.login()
    while True:
        iss = in_q.get(True)
        if iss is None:
            return
        pdf = api.download_issue_pdf(iss.id)
        fn = "%s/%d.pdf" % (path, iss.id)
        out_q.put((fn, pdf))
        
def write(path, raw):
    with open(path, 'wb') as f:
        f.write(raw)
        
        
def write_queue_to_disk(out_q, print_update2):
    # write to disk
    while True:
        try:
            fn, pdf = out_q.get(False)
        except queue.Empty:
            break
        else:
            write(fn, pdf)
        print_update2()
        
        
def download_issues_pdf_multithread(api, issues, path=".", filter_cb=lambda i: True, max_threads=8):
    
    os.makedirs(path, exist_ok=True)
    issues = [i for i in issues if filter_cb(i)]

    in_q = queue.Queue()
    out_q = queue.Queue()
    threads = set()
    
    def print_update():
        nonlocal issues, in_q, max_threads
        nissues = len(issues)
        qsz = in_q.qsize()
        done = nissues - qsz
        if done < 0:
            done = 0  # compensate for sentinels inserted into queue
        print("\rDownloading files with %d threads multithreaded %d/%d           " % (max_threads, done, nissues), end="")
        
    def print_update2():
        nonlocal out_q, issues
        done = len(issues) - out_q.qsize()
        print("\rWriting files to disk: %d/%d         " % (done, len(issues)), end="")
    
    for i in range(max_threads):
        t = threading.Thread(None, download_worker, args=(api.copy(), in_q, out_q, path), daemon=True)
        threads.add(t)
    for t in threads:
        t.start()
    for i in issues:
        in_q.put(i)
    for i in range(max_threads):
        in_q.put(None)
        
    # download stuff
    while True:
        alive = False
        tcopy = list(threads)

        for t in tcopy:
            if not t.is_alive():
                threads.remove(t)

        print_update()
        time.sleep(.5)
        if not threads:
              break
        write_queue_to_disk(out_q, print_update2)
    write_queue_to_disk(out_q, print_update2)
    print()
    print("Done")
        


Writing files to disk: 109/109         
Done


In [619]:
api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
api.login()

<Response [200]>

In [621]:
issues= api.download_issues('pbssoftware', modified_on=">=2016-6-01")
issues = list(issues.values())

def filter_cb(iss):
    return iss.sprint_milestone.name == "3.0" and iss.tracker.name == "Specification"

#path = "pdfs3"
#download_issues_pdf_multithread(i, issues, path, filter_cb)

In [623]:
gantt_issues = api.download_gantt('pbssoftware')

In [591]:
def iter2(subtasks, level):
    for i in subtasks or ():
        yield level+1, i
        for task in iter2(i.subtasks, level+1):
            yield task

def iter_subtasks1(issues):
    issues = [i for i in issues if i.parent is None]
    for i in issues:
        yield 0, i
        for lvl, task in iter2(i.subtasks, 1) or ():
            yield lvl, task
            
def iter_subtasks2(issues, seen=None, level=0):
    if seen is None:
        seen = set()
    for i in issues:
        if level == 0 and i.parent is not None:
            continue
        if i in seen:
            continue
        seen.add(i)
        yield level, i
        yield from iter_subtasks2(i.subtasks, seen, level+1)
    if level == 0:
        print(len(seen), len(issues))

In [659]:
maxlen = max(len(i.subject) for i in gantt_issues)
fmt = "%.20s"
ss = []
issues2 = {i.id for i in issues if i.sprint_milestone.name == "3.0" and i.tracker.name == 'Specification'}
gi = list(iter_subtasks2(gantt_issues))
for lvl, i in gi:
    if i.id not in issues2:
        continue
    link = api._base_url + "/" + "issues/" + str(i.id)
    s="%d,%s,%s\n" % (lvl, i.subject, link)
    ss.append(s)

495 495


In [566]:
def _gantt_duplicate(issue, level):
    raise ValueError("Found duplicate issue at level %d: %r" % (level, issue.subject))

def _map_issues_recursive(issues, seen, level):
    for i in issues:
        if i in seen:
            if level == 0:
                continue
            _gantt_duplicate(issue, level)
        seen.add(i)
        subt = i.subtasks
        _map_issues_recursive(subt, seen, level+1)
    return seen

def _map_issues(issues):
    try:
        seen = _map_issues_recursive(issues, set(), 0)
    except ValueError as e:
        e2 = ValueError()
        e2.args = e.args
        raise e2 from None
    if len(seen) != len(issues):
        raise ValueError("Internal error checking gantt integrity: len(seen) != len(issues).")
    if (set(issues) - seen):
        raise ValueError("Internal error checking gantt integrity: Not all issues seen.")


In [665]:
with open("pdfs3/issue_outline.txt", 'wb') as f:
    f.write(''.join(ss).encode('utf-8'))

In [633]:
i.sprint_milestone

ResourceWithID(name='Future Release', id=33)

In [654]:
next(iter(issues2))

Issue(author=User(name='James Small', id=42), custom_fields={...}, sprint_milestone=ResourceWithID(name='3.0', id=52), category=None, status=ResourceWithID(name='Unit Test', id=10), company=None, created_on=datetime.datetime(2015, 4, 15, 18, 52, 48, tzinfo=tzutc()), description='*FRS2304.0*\r\nFor the web calls and shell actions affected by the permission table, check the user permission before executing the action. If the user does not have permission, do not execute.\r\n\r\nNotes:\r\nThe attached Excel spreadsheet specifies all LabVIEW UI functions, and Server calls, along with their associated permission option.  ', subject='Granular permissions', done_ratio=None, crm_reply_token=None, updated_on=datetime.datetime(2016, 2, 17, 22, 46, 50, tzinfo=tzutc()), id=2304, project=_Project(id=5, name='Software', identifier='pbssoftware', description='This project collects all issues and features in regards to software', parent=None, status=1, is_public=False, custom_fields={'Customer Informa

In [664]:
ss

['1,Login Behavior,https://issue.pbsbiotech.com/issues/989\n',
 '1,Webcontrol.conf file ,https://issue.pbsbiotech.com/issues/1885\n',
 '1,Recipe Steps in Reports are out of order,https://issue.pbsbiotech.com/issues/1262\n',
 '1,Opening Reports in WebUI in Kiosk Mode (Christian placeholder),https://issue.pbsbiotech.com/issues/2309\n',
 '1,File Checksum,https://issue.pbsbiotech.com/issues/2316\n',
 '1,Errors report csv has entries in wrong columns,https://issue.pbsbiotech.com/issues/1263\n',
 '1,Database sometimes gets corrupt,https://issue.pbsbiotech.com/issues/420\n',
 '1,Alarms triggered on RIO power cycle,https://issue.pbsbiotech.com/issues/2522\n',
 '1,Report Generation Bug,https://issue.pbsbiotech.com/issues/2596\n',
 '1,Web Login Improvements (Christian placeholder),https://issue.pbsbiotech.com/issues/2715\n',
 '2,Web UI Login events show "Unknown User",https://issue.pbsbiotech.com/issues/2115\n',
 '2,Web UI Logout unknown user,https://issue.pbsbiotech.com/issues/2445\n',
 '2,End 

In [658]:
for n, iss in gi:
    print(i is iss)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals